# Correlation & (Linear) Fitting

When working with more than one variable we can use various methods to quantify how well the variables correlate with one another. We meaure the correlation between two metrics using a scalar value inclusively between -1 and 1. This value is known as the *correlation coefficient*; values near +1 indicate a strong positive relation, values near -1 indicate a strong negative relation (inversely correlated), and values near or equal to 0 indicate weak or no relation.


## Pearson's Correlation Coefficient

This is the most commonly used correlation coefficient. It is simple to compute, and for us, as users of `numpy`, it is even easier (it is literally just a function call!).

$$
r = \frac{
        n\Sigma{xy}-(\Sigma{x})(\Sigma{y})
    }{
        [\sqrt{n\Sigma{x^{2}}-(\Sigma{x})^{2}][n\Sigma{y^{2}}-(\Sigma{y})^{2}}]
    }
$$

where `r` is our correlation coefficient, `x` and `y` are our data, and `n` is the sample size of the data. It does not *look* too simple at first glance, but it is and is pretty easy to compute. Consider the following dataset showing absences vs final grades for a set of students.

In [ ]:
import pandas as pd
df = pd.DataFrame({
    'absences': [0, 1, 1, 2, 3, 3, 4, 5, 6, 7],
    'grade': [90, 85, 88, 84, 82, 80, 75, 60, 72, 64]
})
df

If we visualize this, we can quickly *see* the correlation between the two metrics, but just how correlated are they? Let's visualize out data and compute the correlation coefficient using the formula above.

In [ ]:
df.plot.scatter(x='absences', y='grade')

A quick reminder that we can reach into a data frame and grab particular columns. These columns are ultimately just `numpy` arrays, and so we can perform arithmetic operations quite easily on them! We are going create dummy variables `x`, `y`, and `n` to make write the code a bit easier.

In [ ]:
n = len(df)
x = df.absences
y = df.grade
n, x, y

While `x` and `y` are specifically `pandas.Series` objects (and not `numpy.ndarry`), we can still utilize them as such, as they mimic the functionality provided by `numpy`. This means we can call functions like `sum` on them to compute their sum easily.

In [ ]:
x.sum(), y.sum()

Lastly, recall that we can apply exponents to values using the `**` operator, e.g. we can sqaure each element of `x` using:

In [ ]:
(x**2)

With all of that situated, let's combine everything together and implement the equation for our correlation coefficient:

In [ ]:
import numpy as np
r = (n * (x * y).sum() - x.sum() * y.sum()) / np.sqrt((n * (x**2).sum() - x.sum()**2) * ((n*(y**2).sum() - y.sum()**2)))
r

The value of `-0.906` implies a strong negative correlation between the two variables (not necessarily causation, though in thise case there is grounds to believe causation). This makes sense - as a student misses school, their grades will likely suffer.

But boo! Why should we need to manually compute this value with all of that code above, especially one that is allegedly so common and popular? Well, luckily `numpy` gives us a pretty easy way to compute this particular metric.

In [ ]:
r = np.corrcoef(df.absences, df.grade)
r

The function [`numpy.corrcoeff`](https://numpy.org/doc/stable/reference/generated/numpy.corrcoef.html) computes the correlation coefficients, and returns to us the *correlation matrix*, which shows the correlation between each variable against every variable, including itself. Each row of this matrix represents the coefficients each variable - the first row is how well it correlates to the set; the second row is how it correlates to the set; etc.The diagonals are 1 since any variable will be strongly correlated to itself. As this can be applied to more than just 2 variables at a time, the generalized form for computing this with `numpy` is:

In [ ]:
r = np.corrcoef([df.absences, df.grade])
r

Note the addition of the square braces. This denotes a *list* of metrics we want to compute the correlation coefficients for, and thus we are not limited to passing just two variables.

There are other means of computing correlation, but for most purposes this is sufficient. Some reminders and things to note about Pearson's Correlation:
 
* this does not indicate causation
* we cannot determine independent/dependent variables
* applicable only to linear relationships
* can be misleading in small sample sizes
* can be skewed due to clusters of data and outliers


## Linear Fitting

Another way we can assess correlation, and as discussed in reference to *Anscombe's Quartet*, is to produce and visualize linear trend lines overtop a scatter of the data. As we have briefly seen before, `numpy` also makes this easy. We are going to make use of `numpy`'s *Polynomial API*.

`numpy` implements a very powerful and flexible [Polynomial API](https://numpy.org/doc/stable/reference/routines.polynomials.html) with many polynomial finding routines implemented. Using a spread of data points we can determine approximate polynomial fittings. For our purposes for looking at correlation, we really only need to use a least squares fit.

We can start by using the `polyfit` function to determine the coefficients of our fitting polynomial:

In [ ]:
import numpy.polynomial.polynomial as polynomial
coeffs = polynomial.polyfit(df.absences, df.grade, 1)
coeffs

These coefficients are for a polynomial of the form:

$$
p(x) = c_{0} + c_{1}x + c_{2}x^{2} + ...
$$

With these coefficients, we can then use the `polyval` function to evaluate the polynomial with these coefficients at the bounds of out data:

In [ ]:
fit = polynomial.polyval([0,7], coeffs)
fit

In [ ]:
ax = df.plot.scatter('absences', 'grade')
ax.plot([0,7], fit)
ax.text(x=4, y=90, s=f'r = {r[0][1]:.3f}', fontsize='large')

# create a nicely formatted polynomial
poly = polynomial.Polynomial([_.round(2) for _ in coeffs])

ax.text(x=4, y=85, s=f'f(x) $\\approx$ {poly}', fontsize='large')

This matches our expectations as we can see a clear trend in the data reinforced with our linear fit

### Where Linear Corrleation & Fitting Does Not Work

As mentioned, there are times when computing the correlation for a data set does not make sense. We are going to take a quick look at some examples of when we want to avoid looking at correlation and linear trends.

#### Nonlinear Data

As correlation is a linear statistic, it is not applicable to use for non-linear data. This does not mean the variables being assessed are not correlated or related in some way, just that we cannot quantify that relationship with the methods learned. Consider quadratic data and its trendline:

In [ ]:
quadratic = pd.DataFrame({
    'x': np.linspace(-5,5,25)
})
quadratic['y'] = quadratic.x**2
ax = quadratic.plot.scatter('x', 'y')

coeffs = polynomial.polyfit(quadratic.x, quadratic.y, 1)
fit = polynomial.polyval([-5,5], coeffs)
ax.plot([-5,5], fit)

The trend line asserts that there is no variability within the data (slope == 0). Now let's compute Pearson's Correlation Coefficient:

In [ ]:
r = np.corrcoef(quadratic.x, quadratic.y)
r

Here we have a corefficient that is essentially 0, though there very well could be a relationship at work here (one that just is not linear!)

#### Small Sample Sizes

This one is applicable to more than just linear correlation - any sample size that is too small is hard to use to draw meaningful conclusions. Consider the following contrived dataset:

In [ ]:
small = pd.DataFrame({
    'x': [1.0, 2.2, 3.8],
    'y': [0.5, 2.2, 2.3]
})
ax = small.plot.scatter('x', 'y')

coeffs = polynomial.polyfit(small.x, small.y, 1)
fit = polynomial.polyval([1.0, 3.8], coeffs)
ax.plot([1.0, 3.8], fit)

Now let's compute Pearson's Correlation Coefficient:

In [ ]:
r = np.corrcoef(small.x, small.y)
r

With our limited data we are told that there is a strong correlation between our variables, but 3 data points is hardly enough to really understand the data. A single additional data point that is within the range of our current data could greatly swing our correlation coefficient in either direction.

#### Clusters of Data

Our data may consist of various clusters of points - these clusters themselves may or may not have trends within, but collectively may show a trend altogether that may be misleading:

In [ ]:
clusters = pd.DataFrame({
    'x': np.concatenate([np.random.uniform(0.0, 0.5, 25), np.random.uniform(5.0, 5.5, 25)]),
    'y': np.concatenate([np.random.uniform(0.0, 0.5, 25), np.random.uniform(5.0, 5.5, 25)]),
})
ax = clusters.plot.scatter('x', 'y')

coeffs = polynomial.polyfit(clusters.x, clusters.y, 1)
fit = polynomial.polyval([0.0, 5.5], coeffs)
ax.plot([0.0, 5.5], fit)

In [ ]:
r = np.corrcoef(clusters.x, clusters.y)
r

There appears to be a very string correlation here in our data, but it is likely more useful to evaluate the individual clusters. 

In [ ]:
clusters = pd.DataFrame({
    'x': np.concatenate([np.random.uniform(0.0, 0.5, 25), np.random.uniform(5.0, 5.5, 25)]),
    'y': np.concatenate([np.random.uniform(0.0, 0.5, 25), np.random.uniform(5.0, 5.5, 25)]),
})
ax = clusters.plot.scatter('x', 'y')

coeffs = polynomial.polyfit(clusters.x, clusters.y, 1)
fit = polynomial.polyval([0.0, 5.5], coeffs)
ax.plot([0.0, 5.5], fit)

In [ ]:
r = np.corrcoef(clusters.x, clusters.y)
r

In [ ]:
cluster1 = clusters.iloc[:25]
ax = cluster1.plot.scatter('x', 'y')

coeffs = polynomial.polyfit(cluster1.x, cluster1.y, 1)
fit = polynomial.polyval([0.0, 0.5], coeffs)
ax.plot([0.0, 0.5], fit)

In [ ]:
r = np.corrcoef(cluster1.x, cluster1.y)
r

In [ ]:
cluster2 = clusters.iloc[25:]
ax = cluster2.plot.scatter('x', 'y')

coeffs = polynomial.polyfit(cluster2.x, cluster2.y, 1)
fit = polynomial.polyval([5.0, 5.5], coeffs)
ax.plot([5.0, 5.5], fit)

In [ ]:
r = np.corrcoef(cluster2.x, cluster2.y)
r

#### Outliers

Outliers in data can greatly skew measures of correlation within a data set.

In [ ]:
outlier = pd.DataFrame({
    'x': np.random.uniform(0.0, 0.5, 25),
    'y': np.random.uniform(0.0, 0.5, 25),
})
outlier.iloc[-1] = [5.0, 5.0]
ax = outlier.plot.scatter('x', 'y')

coeffs = polynomial.polyfit(outlier.x, outlier.y, 1)
fit = polynomial.polyval([0.0, 5.0], coeffs)
ax.plot([0.0, 5.0], fit)

In [ ]:
r = np.corrcoef(outlier.x, outlier.y)
r

Again our trendline and correlation coefficient indicate a strong relation in the data, but the single outlier at `(5.0, 5.0)` is skewing our assessment of the data. If we strip that value from our data, we see that there is not much to our data.

In [ ]:
outlier_removed = outlier.iloc[:-1]
ax = outlier_removed.plot.scatter('x', 'y')

coeffs = polynomial.polyfit(outlier_removed.x, outlier_removed.y, 1)
fit = polynomial.polyval([0.0, 0.5], coeffs)
ax.plot([0.0, 0.5], fit)

In [ ]:
r = np.corrcoef(outlier_removed.x, outlier_removed.y)
r

This is a much more accurate representation of our data! However, it is worth noting that outliers in data are not necessarily bad. There very well could be something in your model or system that behaves outlandish under certain criteria, and while it may make some statistical operations and observations worse/less conclusive, it is always best to understand the reasons behind any and all outliers!